# Mesures d'intérêt des règles d'association

Cet exemple est inspiré et adapté de :  
Jiawei Han, Micheline Kamber, and Jian Pei. *Data Mining: Concepts and Techniques,* Third Edition. The Morgan Kaufmann Series in Data Management Systems (2011): 269-271.

Soit une base de transactions de la forme suivante :

client | café | lait
-------|------|-----
1      |    1 |    0
2      |    1 |    1
3      |    0 |    0
...    |  ... |  ...
1000   |    0 |    1

Pour simplifier l'exercice, ces informations sont représentées de manière synthétique en comptant :
- le nombre de clients achetant du café et du lait (*both*),
- le nombre de clients n'achetant que du café (*coffee*),
- le nombre de clients n'achetant que du lait (*milk*),
- le nombre de clients n'achetant ni lait ni café (*none*).

On considère alors les transactions des six points de vente (PDV) suivants :

PDV |  both | coffee | milk  | none
---:|------:|-------:|------:|-----:
  1 |  1000 |    100 |   100 | 10000
  2 |  1000 |    100 |   100 |    10
  3 |    10 |    100 |   100 | 10000
  4 |   100 |    100 |   100 | 10000
  5 |   100 |     10 |  1000 | 10000
  6 |   100 |      1 | 10000 | 10000


In [1]:
import numpy as np
import pandas as pd
sales = pd.DataFrame({'PDV':    [  '1',  '2',   '3',   '4',   '5',   '6'],
                      'both':   [ 1000, 1000,    10,   100,   100,   100],
                      'coffee': [  100,  100,   100,   100,    10,     1],
                      'milk':   [  100,  100,   100,   100,  1000, 10000],
                      'none':   [10000,   10, 10000, 10000, 10000, 10000]}).set_index('PDV')
sales

,both,coffee,milk,none
PDV,,,,
1,1000,100,100,10000
2,1000,100,100,10
3,10,100,100,10000
4,100,100,100,10000
5,100,10,1000,10000
6,100,1,10000,10000


In [2]:
# pour Q4
ref = '3'
sales = pd.concat([sales] + [sales.loc[ref:ref].rename(index={ref: ref+suffix}) for suffix in ['a', 'b']]).sort_index()
sales.loc[ref+'a', 'none'] = 1000
sales.loc[ref+'b', 'none'] = 11890
sales

,both,coffee,milk,none
PDV,,,,
1,1000,100,100,10000
2,1000,100,100,10
3,10,100,100,10000
3a,10,100,100,1000
3b,10,100,100,11890
4,100,100,100,10000
5,100,10,1000,10000
6,100,1,10000,10000


## Le lift

**Q1** Calculer, pour chaque point de vente :
- P(coffee $\wedge$ milk)
- P(coffee)
- P(coffee | milk)
- P(milk)
- P(milk | coffee)
- le support, la confiance et le lift de la règle d'association coffee $\rightarrow$ milk

### Résultats attendus

Si vous voulez vérifier vos résultats, les valeurs à trouver pour le PDV no. 5 sont les suivantes.

|   PDV | P(coffee $\wedge$ milk) |  P(coffee) | P(coffee &#124; milk) |   P(milk) | P(milk &#124; coffee) |   support | confidence |     lift |
|------:|------------------------:|-----------:|----------------------:|----------:|----------------------:|----------:|-----------:|---------:|
|     5 |               0.0090009 | 0.00990099 |             0.0909091 | 0.0990099 |              0.909091 | 0.0090009 |   0.909091 |  9.18182 |

In [3]:
sales['total'] = sales['both'] + sales['coffee'] + sales['milk'] + sales['none']
sales['sup(c)'] = sales['both'] + sales['coffee']
sales['sup(m)'] = sales['both'] + sales['milk']
sales['P(cm)'] = sales['both'] / sales['total']
sales['P(c)'] = sales['sup(c)'] / sales['total']
sales['P(c|m)'] = sales['both'] / sales['sup(m)']
sales['P(m)'] = sales['sup(m)'] / sales['total']
sales['P(m|c)'] = sales['both'] / sales['sup(c)']
sales['support'] = sales['both'] / sales['total']
sales['confidence'] = sales['P(m|c)']
sales['lift'] = sales['confidence'] / sales['P(m)']
sales

,both,coffee,milk,none,total,sup(c),sup(m),P(cm),P(c),P(c|m),P(m),P(m|c),support,confidence,lift
PDV,,,,,,,,,,,,,,,
1,1000,100,100,10000,11200,1100,1100,0.089286,0.098214,0.909091,0.098214,0.909091,0.089286,0.909091,9.256198
2,1000,100,100,10,1210,1100,1100,0.826446,0.909091,0.909091,0.909091,0.909091,0.826446,0.909091,1.000000
3,10,100,100,10000,10210,110,110,0.000979,0.010774,0.090909,0.010774,0.090909,0.000979,0.090909,8.438017
3a,10,100,100,1000,1210,110,110,0.008264,0.090909,0.090909,0.090909,0.090909,0.008264,0.090909,1.000000
3b,10,100,100,11890,12100,110,110,0.000826,0.009091,0.090909,0.009091,0.090909,0.000826,0.090909,10.000000
4,100,100,100,10000,10300,200,200,0.009709,0.019417,0.500000,0.019417,0.500000,0.009709,0.500000,25.750000
5,100,10,1000,10000,11110,110,1100,0.009001,0.009901,0.090909,0.099010,0.909091,0.009001,0.909091,9.181818
6,100,1,10000,10000,20101,101,10100,0.004975,0.005025,0.009901,0.502463,0.990099,0.004975,0.990099,1.970493


### PDV 1 et 2

**Q2** Si l'on considère que le PDV no. 1 intègre les transactions de son bar et de sa sandwicherie et que le PDV no. 2 ne considère que les transactions de son bar, les transactions des deux PDV laissent-elles supposer des comportements significativement différents entre leurs consommateurs de café ?

## Mesures complémentaires

**Q3** Calculer les mesures suivantes, pour chaque point de vente :
- all_confidence
- max_confidence
- Kulczynski
- cosine
- imbalance ratio
- leverage
- conviction

### Résultats attendus

Si vous voulez vérifier vos résultats, les valeurs à trouver pour le PDV no. 5 sont les suivantes.

|   PDV |  all_conf. | max_conf. |     Kulc. |    cosine |       IR |  leverage | conviction |
|------:|-----------:|----------:|----------:|----------:|---------:|----------:|-----------:|
|     5 | 0.0909091  | 0.909091  | 0.5       | 0.28748   | 0.891892 | 0.0080206 |    9.91089 |

In [4]:
sales['all'] = sales[['P(c|m)', 'P(m|c)']].min(axis=1)
sales['max'] = sales[['P(c|m)', 'P(m|c)']].max(axis=1)
sales['Kulc'] = (sales['P(c|m)'] + sales['P(m|c)']) / 2
sales['cos'] = np.sqrt(sales['P(c|m)'] * sales['P(m|c)'])
sales['IR'] = np.abs(sales['sup(c)'] - sales['sup(m)']) / (sales['both'] + sales['coffee'] + sales['milk'])
sales['leverage'] = sales['P(cm)']-sales['P(c)']*sales['P(m)']
sales['conviction'] = (1 - sales['P(m)']) / (1 - sales['P(m|c)'])
sales.loc[:, 'P(cm)':]

,P(cm),P(c),P(c|m),P(m),P(m|c),support,confidence,lift,all,max,Kulc,cos,IR,leverage,conviction
PDV,,,,,,,,,,,,,,,
1,0.089286,0.098214,0.909091,0.098214,0.909091,0.089286,0.909091,9.256198,0.909091,0.909091,0.909091,0.909091,0.000000,7.963967e-02,9.919643
2,0.826446,0.909091,0.909091,0.909091,0.909091,0.826446,0.909091,1.000000,0.909091,0.909091,0.909091,0.909091,0.000000,1.110223e-16,1.000000
3,0.000979,0.010774,0.090909,0.010774,0.090909,0.000979,0.090909,8.438017,0.090909,0.090909,0.090909,0.090909,0.000000,8.633582e-04,1.088149
3a,0.008264,0.090909,0.090909,0.090909,0.090909,0.008264,0.090909,1.000000,0.090909,0.090909,0.090909,0.090909,0.000000,0.000000e+00,1.000000
3b,0.000826,0.009091,0.090909,0.009091,0.090909,0.000826,0.090909,10.000000,0.090909,0.090909,0.090909,0.090909,0.000000,7.438017e-04,1.090000
4,0.009709,0.019417,0.500000,0.019417,0.500000,0.009709,0.500000,25.750000,0.500000,0.500000,0.500000,0.500000,0.000000,9.331700e-03,1.961165
5,0.009001,0.009901,0.090909,0.099010,0.909091,0.009001,0.909091,9.181818,0.090909,0.909091,0.500000,0.287480,0.891892,8.020604e-03,9.910891
6,0.004975,0.005025,0.009901,0.502463,0.990099,0.004975,0.990099,1.970493,0.009901,0.990099,0.500000,0.099010,0.989902,2.450191e-03,50.251281


### PDV 3

**Q4** Faire varier le nombre de transactions *none* du PDV no. 3 afin d'obtenir un lift inférieur à 1 ou supérieur à 10. Pour cela, il pourra être utile d'exprimer et de calculer la valeur du *lift* en fonction de *both*, *coffee*, *milk* et *none*.

In [5]:
# comparer les PDV 3, 3a et 3b
sales[['none', 'support', 'P(c|m)', 'P(m|c)', 'lift', 'leverage', 'conviction', 'all', 'max', 'Kulc', 'cos', 'IR']]

,none,support,P(c|m),P(m|c),lift,leverage,conviction,all,max,Kulc,cos,IR
PDV,,,,,,,,,,,,
1,10000,0.089286,0.909091,0.909091,9.256198,7.963967e-02,9.919643,0.909091,0.909091,0.909091,0.909091,0.000000
2,10,0.826446,0.909091,0.909091,1.000000,1.110223e-16,1.000000,0.909091,0.909091,0.909091,0.909091,0.000000
3,10000,0.000979,0.090909,0.090909,8.438017,8.633582e-04,1.088149,0.090909,0.090909,0.090909,0.090909,0.000000
3a,1000,0.008264,0.090909,0.090909,1.000000,0.000000e+00,1.000000,0.090909,0.090909,0.090909,0.090909,0.000000
3b,11890,0.000826,0.090909,0.090909,10.000000,7.438017e-04,1.090000,0.090909,0.090909,0.090909,0.090909,0.000000
4,10000,0.009709,0.500000,0.500000,25.750000,9.331700e-03,1.961165,0.500000,0.500000,0.500000,0.500000,0.000000
5,10000,0.009001,0.090909,0.909091,9.181818,8.020604e-03,9.910891,0.090909,0.909091,0.500000,0.287480,0.891892
6,10000,0.004975,0.009901,0.990099,1.970493,2.450191e-03,50.251281,0.009901,0.990099,0.500000,0.099010,0.989902


**Q5** Qu'indiquent les autres critères sur la qualité de la règle coffee $\rightarrow$ milk pour ce PDV ? Lesquels varient ?

> À 1000, le lift vaut exactement 1 et à 11890, il vaut exactement 10.
> 
> On peut résoudre analytiquement avec la formule suivante :
> - $l$ : lift
> - $b$ : both
> - $c$ : coffee
> - $m$ : milk
> - $n$ : none
> 
> $$n = \frac{l(b + m)(b + c)}{b}-(b+m+c)$$
> 
> Le PDV no. 3 illustre le fait qu'en augmentant le nombre de transactions non pertinentes, on peut obtenir un lift satisfaisant pour une règle considérée comme mauvaise par tous les autres critères (support, confiance, all_confidence, max_confidence, Kulc et cosine).  
On observe le même comportement pour le leverage, tandis que la conviction nous donne une indication, pour les trois cas, que coffee et $\neg$milk sont (quasiment) indépendants.

### PDV 4, 5 et 6

Les PDV no. 4, 5 et 6 présentent des associations (coffee $\rightarrow$ milk) et (milk $\rightarrow$ coffee) de plus en plus déséquilibrées :  
la confiance de la règle coffee $\rightarrow$ milk &ndash; i.e. P(milk | coffee) &ndash; augmente alors que P(coffee | milk) diminue.  
C'est-à-dire que les buveurs de café prennent de plus en plus systématiquement du lait, alors que les buveurs de lait en général prennent de moins en moins de café.

**Q6** Comme les mesures lift, all_conf., max_conf, Kulc. et cosine sont symétriques, ces résultats ne donnent-ils pas des informations relativement contradictoires selon la règle d'association considérée : (coffee $\rightarrow$ milk) ou (milk $\rightarrow$ coffee) ?

**Q7** Les auteurs de (Han et al., 2011) recommandent de considérer à la fois la mesure de Kulczynski et l'imbalance ratio. Cette solution vous semble-t-elle raisonnable ?  
Plus précisément, si une mesure de Kulczynski est proche de 0 ou de 1, alors la règles est intéressante. Au contraire, si la mesure de Kulczynski est proche de 0.5, alors la règle n'est intéressante que si l'IR est élevé (proche de 1).

> Il est donc souhaitable de prendre en compte le déséquilibre entre les deux itemset lors de l'interprétation des mesures. L'*imbalance ratio* (IR) permet de quantifier ce déséquilibre.
> 
> Un Kulc de 0.5 et un IR de 0 représente une situation d'indépendance.
> Un Kulc de 0.5 et un IR élevé indique que la règle possède une confiance très faible ou très élevée en fonction du sens de lecture.
> 
> Globalement, la mesure considérée peut dépendre du cadre applicatif, et en particulier du sens que l'on donne aux transactions non pertinentes : le sont-elles vraiment ?

## Vérification avec mlxtend

Il est possible de *construire* des itemsets fréquents pour analyse. En particulier si vous manipulez des données agrégées.

**Q8** Construire, pour un des PDV, une DataFrame avec les informations de support des itemsets {café}, {lait} et {café, lait}.

In [6]:
# Exemple pour le PDV no. 5
fq = pd.DataFrame({
    'support': [100/11110, 110/11110, 1100/11110],
    'itemsets': [('coffee', 'milk'), ('coffee',), ('milk',)]
})
fq

,support,itemsets
0,0.009001,"(coffee, milk)"
1,0.009901,"(coffee,)"
2,0.099010,"(milk,)"


**Q9** Calculer les règles d'association sur cette DataFrame.

In [7]:
from mlxtend.frequent_patterns import association_rules

In [8]:
association_rules(fq, min_threshold=0.001)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(milk),(coffee),0.099010,0.009901,0.009001,0.090909,9.181818,0.008021,1.089109
1,(coffee),(milk),0.009901,0.099010,0.009001,0.909091,9.181818,0.008021,9.910891
